In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('meta/meta.txt', sep = '\t', header=None)

In [4]:
data.shape

(11307, 5)

In [5]:
data.head()

,0,1,2,3,4
0,background_0001.wav,office,0,9.458500,background
1,background_0001_time_stretch_0.wav,office,0,9.989070,background
2,background_0001_time_stretch_1.wav,office,0,9.879410,background
3,background_0001_time_stretch_10.wav,office,0,11.156553,background
4,background_0001_time_stretch_11.wav,office,0,13.261587,background


In [6]:
data.columns = ['name', 'place', 'start', 'end', 'label']

In [7]:
data.head()

,name,place,start,end,label
0,background_0001.wav,office,0,9.458500,background
1,background_0001_time_stretch_0.wav,office,0,9.989070,background
2,background_0001_time_stretch_1.wav,office,0,9.879410,background
3,background_0001_time_stretch_10.wav,office,0,11.156553,background
4,background_0001_time_stretch_11.wav,office,0,13.261587,background


In [8]:
data.label.value_counts()

door             3416
tool             1659
knocking_door    1656
bags             1236
keyboard         1225
background       1126
ring              713
speech            276
Name: label, dtype: int64

In [10]:
data.describe()

,start,end
count,11307.0,11307.000000
mean,0.0,5.351488
std,0.0,9.689636
min,0.0,0.123447
25%,0.0,1.898299
50%,0.0,3.296190
75%,0.0,4.739550
max,0.0,327.412000


In [8]:
from python_speech_features import mfcc
from python_speech_features import delta
from python_speech_features import logfbank
import scipy.io.wavfile as wav



In [55]:

def create_ceps(path,fn):
    (rate,sig) = wav.read(path+fn)
    mfcc_feat = mfcc(sig,rate,nfft=551)
    base_fn, ext = os.path.splitext(fn)
    data_fn = 'ceps/' + base_fn + '.ceps'
    scipy.save(data_fn, mfcc_feat)
    #d_mfcc_feat = delta(mfcc_feat, 2)
    #fbank_feat = logfbank(sig,rate)
    #print(fbank_feat[1:3,:])

    

In [18]:
for i in range(data.shape[0]):
    create_ceps(data.name[i])
    

In [22]:
def read_ceps():
    X, y = [],[]
    for i in range(data.shape[0]):
        base_fn, ext = os.path.splitext(data.name[i])
        fn = 'ceps/' + base_fn + '.ceps.npy'
        ceps = np.load(fn)
        num_ceps = len(ceps)
        X.append(np.mean(ceps[int(num_ceps*0.1):int(num_ceps*0.9)], axis=0))
        y.append(data.label[i])
    return np.array(X), np.array(y)

In [23]:
train, target = read_ceps()

In [24]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import train_test_split

C:\Users\User\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [25]:
neighbors = KNeighborsClassifier(n_neighbors=5)

In [26]:
trainX, testX, trainY, testY = train_test_split(train, target, test_size = 0.3)
neighbors.fit(trainX, trainY)
print('Accuracy: \n', neighbors.score(testX, testY))
pred = neighbors.predict(testX)

('Accuracy: \n', 0.9717064544650752)


In [53]:
test_files = os.listdir('test/') 

In [56]:
for t in test_files:
    create_ceps('test/',t)

In [57]:
def read_test_ceps():
    X = []
    for t in test_files:
        base_fn, ext = os.path.splitext(t)
        fn = 'ceps/' + base_fn + '.ceps.npy'
        ceps = np.load(fn)
        num_ceps = len(ceps)
        X.append(np.mean(ceps[int(num_ceps*0.1):int(num_ceps*0.9)], axis=0))
    return np.array(X)

In [58]:
test = read_test_ceps()

In [59]:
test.shape

(610L, 13L)

In [73]:
np.max(neighbors.predict_proba(t))

0.4

In [77]:
with open('result.txt', "w") as fout:
    for i in range(test.shape[0]):
        t = test[i].reshape(1, -1)
        score = np.max(neighbors.predict_proba(t))
        class_label = neighbors.predict(t)[0]
        line = test_files[i] + ' ' + str(score) + ' ' + class_label + '\n'
        fout.write(line)